In [1]:
!pip install -q py-feat
# !pip install -q facial-emotion-recognition
# !pip install -q fer
# !pip install -q deepface
# !pip install -q mtcnn

     |████████████████████████████████| 6.0MB 7.1MB/s 
     |████████████████████████████████| 3.3MB 37.0MB/s 
     |████████████████████████████████| 57.4MB 56kB/s 
     |████████████████████████████████| 4.9MB 45.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from feat import Detector
import glob
import math
import dlib
import joblib
import cv2
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from collections import Counter

import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

face_model = "RetinaFace"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

def my_mode(sample):
  c = Counter(sample)
  return [k for k, v in c.items() if v == c.most_common(1)[0][1]]

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


Loading Face Detection model:  RetinaFace



Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib



Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_scalar_aus.joblib



Loading Face Landmark model:  MobileNet
Loading au model:  svm
Loading emotion model:  resmasknet


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## ALL STEP IN ONE

In [3]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

# Create  model
model_4 = Model(inputs=base_model.input, outputs=output)

base_model_layers = model_4.layers[:23]
new_model_layers =  model_4.layers[23:]

base_model_blocks = {
    0: base_model_layers[1:3],
    1: base_model_layers[4:6],
    2: base_model_layers[7:11],
    3: base_model_layers[12:16],
    4: base_model_layers[17:21]
}

# Load the best model
model_4_path = "./drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/model_ck_kaggle_jaffe/models/{}.h5".format("model_4_s_alltrain")
model_4.load_weights(model_4_path)

80150528/80134624 [==============================] - 0s 0us/step


In [4]:
ls 'drive/MyDrive/file2/vdo/Monsrung'

01_mintchyy.mp4  02_babyjingko.mp4


In [5]:
ls 'drive/MyDrive/file2/done/'

face2_01_KARNMAY_P1.csv          face6_01_mintchyy_ep2.xlsx       face9_2.xlsx
face2_01_KARNMAY.xlsx            face6_02_Babyjingko_P1.csv       face9_3.csv
face2_02_Kemisara_P1.csv         face6_02_Babyjingko.xlsx         face9_3.mp4
face2_02_Kemisara.xlsx           face7_01_mintchyy_QA_100_P1.csv  face9_3.xlsx
face5_01_jaysbabyfood_P1.csv     face7_01_mintchyy_QA_100.xlsx    face9_4.csv
face5_01_jaysbabyfood_P1.gsheet  face_7_2_01_thematter_P1.csv     face9_4.mp4
face5_01_jaysbabyfood.xlsx       face_7_2_01_thematter.xlsx       face9_4.xlsx
face5_02_gssspotted_P1.csv       face9_1.csv                      face9_5.csv
face5_02_gssspotted.xlsx         face9_1.mp4                      face9_5.mp4
face5_03_clubfriday_P1.csv       face9_1.xlsx                     face9_5.xlsx
face5_03_clubfriday.xlsx         face9_2.csv                      old/
face6_01_mintchyy_ep2_P1.csv     face9_2.mp4


In [6]:
pred_emotion= []

filepath = 'drive/MyDrive/file2/done/face9_5.csv'
vdopath = 'drive/MyDrive/file2/done/face9_5.mp4'
df = pd.read_csv(filepath)
dim = (224, 224)
rowimg = []
label_img = []
print('df', df.shape)

for i in tqdm(range(df.shape[0])):
  seqimg = []
  for j in range(df['start_frame'][i], df['end_frame'][i]):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    try:
      pos = faceimg[0][:4]
      cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] 
      resized = cv2.resize(cropimg, dim, interpolation=cv2.INTER_CUBIC)     
      image = preprocess_input(resized)
      image = np.expand_dims(image, axis=0)
      preds = model_4.predict(image)
      pred_class = np.argmax(preds)
      seqimg.append(pred_class) 
    except:
      print("Emotion Error  i = ",i, ' j =', j)

  seq_label = my_mode(seqimg)
  print('i =', i, 'seq_label =', seq_label)
  try:
    pred_emotion.append(seq_label[0])
  except:
    pass

df (22, 7)


i = 0 seq_label = [2]
i = 1 seq_label = [2]
i = 2 seq_label = [1]
i = 3 seq_label = [2]
i = 4 seq_label = [2]
i = 5 seq_label = [1]
i = 6 seq_label = [1]
i = 7 seq_label = [2]
i = 8 seq_label = [1]
i = 9 seq_label = [1]
i = 10 seq_label = [0]
i = 11 seq_label = [1]
i = 12 seq_label = [2]
i = 13 seq_label = [1]
i = 14 seq_label = [2]
i = 15 seq_label = [2]
i = 16 seq_label = [0]
i = 17 seq_label = [2]
i = 18 seq_label = [1]
i = 19 seq_label = [1]
i = 20 seq_label = [1]
i = 21 seq_label = [1]



In [ ]:
len(pred_emotion)

In [7]:
df = pd.DataFrame(np.array(pred_emotion), columns=['label'])
from google.colab import files
df.to_csv('face9_5_r.csv') 
files.download('face9_5_r.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
while True:pass